In [34]:
# import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
import xgboost as xgb

import math
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import spearmanr

path = 'C:\\Users\\mrgba\\OneDrive\\Thesis\\ThesisData\\RawData\\'

In [35]:
# define evaluation metrics
def MAPE(test_y, pred_y): 
    test_y, pred_y = np.array(test_y), np.array(pred_y)
    return np.mean(np.abs((test_y - pred_y) / test_y)) * 100

# model must be fit on training data before using evaluate
def evaluate(model, test_x, test_y):
    pred_y = model.predict(test_x)
    rmse = MSE(test_y, pred_y, squared = False)
    mape = MAPE(test_y, pred_y)
    pearson = pearsonr(test_y, pred_y)
    spearman = spearmanr(test_y, pred_y)
    return rmse, mape, pearson, spearman, pred_y

In [36]:
# define lists of things to be iterated through for each loop

# LOOP 1
# outermost loop iterates through all four different prediction methods
methods = ['M1', 'M2', 'M3', 'M4']

# LOOP 2
# next loop iterates through allowing the use of aircraft data or not.
# allowing aircraft data will be denoted as 'all', where not allowing it
# will only use subject-related features, denoted as 'subj'
datatype = ['all', 'subj']

# LOOP 3
# next loop iterates through all four different types of models being compared
# RandomForest = 'rf', ExtraTrees = 'xt', XGBoost = 'xgb', CategoricalBoosting = 'cb'
modeltype = ['rf', 'xt', 'xgb', 'cb']

# also need a list of dictionaries, where each dictionary is the different hyperparameters
# to be explored through the cross-validated grid search
hyperparameters = [
    # RandomForest Hyperparameters
    {'bootstrap': [False, True],
     'max_depth' : [2,3,4,5,6,7,None],
     'min_samples_leaf': [1,2,3,4,5,6],
     'min_samples_split': [2,3,4,5,6,7],
     'n_estimators': [10,25,50,100,150,200]}, 
    # ExtraTrees Hyperparameters
    {'n_estimators' : [10,25,50,100,150,200],
     'max_depth' : [2,3,4,5,6,None],
     'min_samples_split' : [2,3,4,5,6,7],
     'min_samples_leaf' : [1,2,3,4,5,6],
     'bootstrap' : [True, False]},
    # XGBoost Hyperparameters
    {'max_depth': [2,3,4,5,6,7,None],
     'n_estimators': [10,25,50,100,150,200],
     'colsample_bytree': [0.1,0.15,0.2,0.25,.3,0.4,0.5],
     'booster' : ['gbtree','dart']},
    # CatBoost Hyperparameters
    {'depth': [2,3,4,5],
     'l2_leaf_reg' : [2,6,4,8,10],
     'learning_rate' : [0.01,0.02,0.03,0.04,0.05],
     'min_child_samples' : [1,2,4,6,8]}
]

# LOOP 4
# next loop performs 10 repetitions of each model, where the list is different random seeds
seeds = [42, 4, 32, 2, 7, 2021, 2013, 97, 835, 13]

# need a list of the different error types to be pulled for Y data
errortype = ['Cumulative_Error', 'GlideSlope_Error', 'Localizer_Error', 'Airspeed_Error']
# with respect to each error source above, need a list of lists of all candidate values to be explored.
# conservatively took the min/max of each error source, building a dense candidate list of every 100 between them
candidates = [np.arange(700, 12100, 100).tolist(), # Cumulative Error candidates values
              np.arange(100, 5100, 100).tolist(), # GlideSlope Error candidates values
              np.arange(0, 6100, 100).tolist(), # Localizer Error candidates values
              np.arange(100, 5100, 100).tolist()] # Airspeed Error candidates values

In [37]:
# define necessary functions for operational use in code

def DATASORT(data_type, method):
    
    x_train = pd.read_csv(path+'trainX_all.csv')
    x_test = pd.read_csv(path+'testX_all.csv')
    features = pd.read_csv(path+'selected_features.csv')
    
    if method == 'M1':
        # only need features selected for cumulative error for these two methods
        if data_type == 'subj':
            # need to remove NaNs from column
            feats = list(features['subj_Cumulative_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train = x_train[cleanedList]
            x_test = x_test[cleanedList]    
        else:
            feats = list(features['all_Cumulative_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train = x_train[cleanedList]
            x_test = x_test[cleanedList]
        
        return x_train, x_test
    
    elif method == 'M3':
        # need to get all unique features selected for the three individual error sources
        if data_type == 'subj':
            # need to remove NaNs from column
            feats1 = list(features['subj_GlideSlope_Error_Features'].values)
            feats2 = list(features['subj_Localizer_Error_Features'].values)
            feats3 = list(features['subj_Airspeed_Error_Features'].values)
            feats = feats1+feats2+feats3
            cleanedList = [x for x in feats if str(x) != 'nan']
            # remove duplicates
            cleanerList = [*set(cleanedList)]
            x_train = x_train[cleanerList]
            x_test = x_test[cleanerList]    
        else:
            feats1 = list(features['all_GlideSlope_Error_Features'].values)
            feats2 = list(features['all_Localizer_Error_Features'].values)
            feats3 = list(features['all_Airspeed_Error_Features'].values)
            feats = feats1+feats2+feats3
            cleanedList = [x for x in feats if str(x) != 'nan']
            # remove duplicates
            cleanerList = [*set(cleanedList)]
            x_train = x_train[cleanerList]
            x_test = x_test[cleanerList] 
        
        return x_train, x_test
    
    # though same dataset for method 4, returning multiple to match formatting for model training/combination    
    elif method == 'M4':
        if data_type == 'subj':
            # GlideSlope
            feats = list(features['subj_Cumulative_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_g = x_train[cleanedList]
            x_test_g = x_test[cleanedList]
            # Localizer
            feats = list(features['subj_Cumulative_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_l = x_train[cleanedList]
            x_test_l = x_test[cleanedList]
            # Airspeed
            feats = list(features['subj_Cumulative_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_a = x_train[cleanedList]
            x_test_a = x_test[cleanedList]
        else:
            # GlideSlope
            feats = list(features['all_Cumulative_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_g = x_train[cleanedList]
            x_test_g = x_test[cleanedList]
            # Localizer
            feats = list(features['all_Cumulative_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_l = x_train[cleanedList]
            x_test_l = x_test[cleanedList]
            # Airspeed
            feats = list(features['all_Cumulative_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_a = x_train[cleanedList]
            x_test_a = x_test[cleanedList] 
        
        return x_train_g, x_test_g, x_train_l, x_test_l, x_train_a, x_test_a
        
        
    # finally, method 2 datasets
    else:
        # M2
        if data_type == 'subj':
            # GlideSlope
            feats = list(features['subj_GlideSlope_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_g = x_train[cleanedList]
            x_test_g = x_test[cleanedList]
            # Localizer
            feats = list(features['subj_Localizer_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_l = x_train[cleanedList]
            x_test_l = x_test[cleanedList]
            # Airspeed
            feats = list(features['subj_Airspeed_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_a = x_train[cleanedList]
            x_test_a = x_test[cleanedList]
        else:
            # GlideSlope
            feats = list(features['all_GlideSlope_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_g = x_train[cleanedList]
            x_test_g = x_test[cleanedList]
            # Localizer
            feats = list(features['all_Localizer_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_l = x_train[cleanedList]
            x_test_l = x_test[cleanedList]
            # Airspeed
            feats = list(features['all_Airspeed_Error_Features'].values)
            cleanedList = [x for x in feats if str(x) != 'nan']
            x_train_a = x_train[cleanedList]
            x_test_a = x_test[cleanedList] 
        
        return x_train_g, x_test_g, x_train_l, x_test_l, x_train_a, x_test_a

In [38]:
# define function for transductive conformal prediction

def TCP(x_train, x_test, y_train, y_test, cand_list, model, alpha):
    
    coverage_count = 0
    width = []
    lower = []
    upper = []
    
    g = 0

    while g < len(x_test):

        # pull row of data points, append to training data set
        x_nplus_1 = x_test.iloc[g:g+1,:]
        x_train_agg = pd.concat([x_train, x_nplus_1], ignore_index = True)

        z = 0
        vals_in_interval = []
        # iterate through all candidate values

        while z < len(cand_list):

            # pull candidate value and append it to y values
            y_c = pd.Series([cand_list[z]],index=[0])
            y_train_agg = pd.concat([y_train, y_c], ignore_index = True)

            new_model = model

            new_model = new_model.fit(x_train_agg, y_train_agg)
            preds = new_model.predict(x_train_agg)

            conformity_scores = abs(y_train_agg - preds)

            r_nplus_1 = conformity_scores.loc[len(conformity_scores)-1]
            indicator_func = ((conformity_scores <= r_nplus_1).astype(int)).sum() - 1
            pi_y_nplus_1 = (1/len(y_train_agg)) + ((1/len(y_train_agg))*indicator_func)

            if (len(y_train_agg)*pi_y_nplus_1) <= math.ceil((1-alpha)*len(y_train_agg)):
                vals_in_interval.append(cand_list[z])

            z += 1

        width.append(max(vals_in_interval) - min(vals_in_interval))
        
        if (y_test.iloc[g] <= max(vals_in_interval)) and (y_test.iloc[g] >= min(vals_in_interval)):
            coverage_count += 1
        lower.append(min(vals_in_interval))
        upper.append(max(vals_in_interval))
        
        g += 1
    
    avg_pi_width = np.mean(width)
    stddev_pi_width = np.std(width)
    marg_cov = coverage_count/(len(x_test))
    
    return avg_pi_width, stddev_pi_width, marg_cov, lower, upper


In [ ]:
# build large dataframe to handle all results
results = pd.DataFrame(columns=['Method','DataType','ErrorSource','ModelType','Iteration',
                                'RMSE','MAPE','Pearson','Spearman','AvgPIWidth','StdDevPIWidth',
                                'MarginalCoverage','Hyperparameters'])

# reading in data for use with aggregation
true_cumul = pd.read_csv(path+'testY.csv')[errortype[0]]

# begin first loop, all four methods
for i in methods:    
# begin second loop, all or subj data used
    for j in datatype:       
# load training/testing data here so that it wont have to be done multiple times in the following loops
        if i == 'M1' or i == 'M3':
            x_train, x_test = DATASORT(j, i)
            y_train = pd.read_csv(path+'trainY.csv')[errortype[0]]
            y_test = pd.read_csv(path+'testY.csv')[errortype[0]]
        else:
            x_train_g, x_test_g, x_train_l, x_test_l, x_train_a, x_test_a = DATASORT(j, i)
            y_train_g = pd.read_csv(path+'trainY.csv')[errortype[1]]
            y_test_g = pd.read_csv(path+'testY.csv')[errortype[1]]
            y_train_l = pd.read_csv(path+'trainY.csv')[errortype[2]]
            y_test_l = pd.read_csv(path+'testY.csv')[errortype[2]]
            y_train_a = pd.read_csv(path+'trainY.csv')[errortype[3]]
            y_test_a = pd.read_csv(path+'testY.csv')[errortype[3]]        
# begin third loop, each of the four different models
        for k in modeltype:            
# begin fourth loop, 10 iterations of the model with different seeds
            for l in range(len(seeds)):
                if k == 'rf':
                    model = RandomForestRegressor(random_state = seeds[l])
                    param_grid = hyperparameters[0]    
                elif k == 'xt':
                    model = ExtraTreesRegressor(random_state = seeds[l])
                    param_grid = hyperparameters[1]    
                elif k == 'xgb':
                    model = xgb.XGBRegressor(objective='reg:squarederror', seed = seeds[l])
                    param_grid = hyperparameters[2]
                else:
                    model = CatBoostRegressor(random_state = seeds[l], verbose=0)
                    param_grid = hyperparameters[3]
                    
                grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 5, verbose = 2, n_jobs = -1)
                if i == 'M1' or i == 'M3':
                    grid_search.fit(x_train, y_train)
                    best_parameters = grid_search.best_params_
                    best_model = grid_search.best_estimator_
                    best_model.fit(x_train, y_train)
                    rmse, mape, pearson, spearman, pred_y = evaluate(best_model, x_test, y_test)
                    
                    # Now perform transductive conformal prediction
                    avg_pi_width, stddev_pi_width, marg_cov, lower, upper = TCP(x_train, x_test, y_train, y_test,
                                                 candidates[0], grid_search.best_estimator_, 0.05)
                    
                    # finally, add all results to dataframe
                    results.loc[len(results)] = [i,j,'Cumulative',k,l+1,
                                                 rmse,mape,pearson,spearman,avg_pi_width,stddev_pi_width,
                                                 marg_cov,best_parameters]
                    
                else:
                    # Need aggregated techniques for methods 2 and 4
                    # GlideSlope Error
                    grid_search.fit(x_train_g, y_train_g)
                    best_parameters = grid_search.best_params_
                    best_model = grid_search.best_estimator_
                    best_model.fit(x_train_g, y_train_g)
                    rmse, mape, pearson, spearman, pred_y_g = evaluate(best_model, x_test_g, y_test_g)
                    
                    # Now perform transductive conformal prediction
                    avg_pi_width, stddev_pi_width, marg_cov, lower_g, upper_g = TCP(x_train_g, x_test_g, y_train_g, y_test_g,
                                                     candidates[1], grid_search.best_estimator_, 0.017)
                    
                    # finally, add all results to dataframe
                    results.loc[len(results)] = [i,j,'GlideSlope',k,l+1,
                                                 rmse,mape,pearson,spearman,avg_pi_width,stddev_pi_width,
                                                 marg_cov,best_parameters]
                    
                    # Localizer Error
                    grid_search.fit(x_train_l, y_train_l)
                    best_parameters = grid_search.best_params_
                    best_model = grid_search.best_estimator_
                    best_model.fit(x_train_l, y_train_l)
                    rmse, mape, pearson, spearman, pred_y_l = evaluate(best_model, x_test_l, y_test_l)
                    
                    # Now perform transductive conformal prediction
                    avg_pi_width, stddev_pi_width, marg_cov, lower_l, upper_l = TCP(x_train_l, x_test_l, y_train_l, y_test_l,
                                                     candidates[2], grid_search.best_estimator_, 0.017)
                    
                    # finally, add all results to dataframe
                    results.loc[len(results)] = [i,j,'Localizer',k,l+1,
                                                 rmse,mape,pearson,spearman,avg_pi_width,stddev_pi_width,
                                                 marg_cov,best_parameters]
                    
                    # Airspeed Error
                    grid_search.fit(x_train_a, y_train_a)
                    best_parameters = grid_search.best_params_
                    best_model = grid_search.best_estimator_
                    best_model.fit(x_train_a, y_train_a)
                    rmse, mape, pearson, spearman, pred_y_a = evaluate(best_model, x_test_a, y_test_a)
                    
                    # Now perform transductive conformal prediction
                    avg_pi_width, stddev_pi_width,
                    marg_cov, lower_a, upper_a = TCP(x_train_a, x_test_a, y_train_a, y_test_a,
                                                     candidates[3], grid_search.best_estimator_, 0.017)
                    
                    # finally, add all results to dataframe
                    results.loc[len(results)] = [i,j,'Airspeed',k,l+1,
                                                 rmse,mape,pearson,spearman,avg_pi_width,stddev_pi_width,
                                                 marg_cov,best_parameters]
                    
                    # Use lower and uppers from individual predictions intervals to combine for aggregated prediction intervals
                    
                    agg_pred = pred_y_g + pred_y_l + pred_y_a
                    agg_lower = lower_g + lower_l + lower_a
                    agg_upper = upper_g + upper_l + upper_a
                    agg_width = agg_upper - agg_lower
                    
                    agg_cov_count = 0
                    for h in range(len(true_cumul)):
                        if true_cumul.iloc[h] <= agg_upper[h] and true_cumul.iloc[h] >= agg_lower[h]:
                            agg_cov_count += 1
                    marg_cov = agg_cov_count / len(true_cumul)
                    avg_pi_width = np.mean(agg_width)
                    stddev_pi_width = np.std(agg_width)
                    rmse = MSE(agg_pred, agg_pred, squared = False)
                    mape = MAPE(agg_pred, agg_pred)
                    pearson = pearsonr(agg_pred, agg_pred)
                    spearman = spearmanr(agg_pred, agg_pred)
                    best_parameters = 'N/A'
                    
                    # add aggregated results to dataframe for approximated cumulative error
                    results.loc[len(results)] = [i,j,'Cumulative',k,l+1,
                                                 rmse,mape,pearson,spearman,avg_pi_width,stddev_pi_width,
                                                 marg_cov,best_parameters]
            print(results)

# automation complete! Save finalized results
results.to_csv(path+'results.csv', encoding='utf-8', header='true', index = False)

Fitting 5 folds for each of 3024 candidates, totalling 15120 fits
Fitting 5 folds for each of 3024 candidates, totalling 15120 fits
